<a href="https://colab.research.google.com/github/seulmi0827/fininsight/blob/main/JACE/Kappa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


출력값에 대한 설명입니다:

Cohen's Kappa: 0.8603

두 평가자 간의 일치도가 우연 요소를 제외하고 86.03%라는 의미입니다.
0.81-1.00 범위에 있어 '거의 완벽한(almost perfect)' 일치도를 나타냅니다.


관찰된 일치도: 0.9655

두 평가자가 실제로 96.55%의 케이스에서 같은 판단을 내렸습니다.


고유 값: [0. 1.]

두 평가자가 사용한 값은 0과 1뿐입니다(이진분류).


값 0.0: 평가자1 비율 0.1379, 평가자2 비율 0.1502

평가자1은 전체 중 13.79%를 0으로 분류했습니다.
평가자2는 전체 중 15.02%를 0으로 분류했습니다.


값 1.0: 평가자1 비율 0.8621, 평가자2 비율 0.8498

평가자1은 전체 중 86.21%를 1로 분류했습니다.
평가자2는 전체 중 84.98%를 1로 분류했습니다.


우연에 의한 일치도: 0.7533

두 평가자가 무작위로 판단했을 때 우연히 일치할 확률이 75.33%입니다.
계산: (0.1379×0.1502) + (0.8621×0.8498) = 0.7533


직접 계산한 Kappa: 0.8603

공식으로 직접 계산한 kappa 값으로 scikit-learn 결과와 일치합니다.
계산: (0.9655-0.7533)/(1-0.7533) = 0.8603



결론적으로, 두 평가자는 우연의 일치를 고려하더라도 매우 높은 수준의 일치도(0.8603)를 보여주고 있습니다.

# CSV

In [ ]:
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import numpy as np
import zipfile
import os

def calculate_kappa_statistics(rater1_clean, rater2_clean):

    # Cohen's kappa 계산
    kappa = cohen_kappa_score(rater1_clean, rater2_clean)

    # 관찰된 일치도 계산
    observed_agreement = sum(r1 == r2 for r1, r2 in zip(rater1_clean, rater2_clean)) / len(rater1_clean)

    # 고유 값 확인
    unique_values = np.unique(np.concatenate([rater1_clean.unique(), rater2_clean.unique()]))

    # 각 값의 비율 계산
    value_ratios = {}
    for val in unique_values:
        p_val = sum(rater1_clean == val) / len(rater1_clean)
        q_val = sum(rater2_clean == val) / len(rater2_clean)
        value_ratios[val] = {'rater1': p_val, 'rater2': q_val}

    # 우연에 의한 일치도 계산
    chance_agreement = sum(
        sum(rater1_clean == val) / len(rater1_clean) * sum(rater2_clean == val) / len(rater2_clean)
        for val in unique_values
    )

    # 직접 kappa 계산
    manual_kappa = (observed_agreement - chance_agreement) / (1 - chance_agreement)

    return {
        'kappa': kappa,
        'observed_agreement': observed_agreement,
        'unique_values': unique_values,
        'value_ratios': value_ratios,
        'chance_agreement': chance_agreement,
        'manual_kappa': manual_kappa
    }
def analyze_agreement(path1, path2, output_prefix="", create_zip=True):
    # CSV 파일 읽기
    rater1_df = pd.read_csv(path1)
    rater2_df = pd.read_csv(path2)

    # 첫번째 평가자와 두번째 평가자의 평가 데이터 추출
    rater1 = rater1_df.iloc[:, 1]
    rater2 = rater2_df.iloc[:, 1]

    # 결측값이 없는 유효한 데이터만 선택
    valid_indices = ~(rater1.isna() | rater2.isna())

    # 유효한 행만 필터링 (loc 사용)
    filtered_df = rater1_df.loc[valid_indices].copy()

    # 평가 데이터 가져오기
    rater1_clean = rater1[valid_indices].astype(float)
    rater2_clean = rater2[valid_indices].astype(float)

    # 카파 통계 계산
    stats = calculate_kappa_statistics(rater1_clean, rater2_clean)

    # 새 데이터프레임 생성
    full_df = pd.DataFrame()

    # 첫 번째 열에 "단어" 추가 (원본 데이터에 있는 경우)
    if len(filtered_df.columns) > 0:
        full_df['단어'] = filtered_df.iloc[:, 0].reset_index(drop=True)

    full_df['rater1'] = rater1_clean.reset_index(drop=True)
    full_df['rater2'] = rater2_clean.reset_index(drop=True)

    # 분류 조건에 따라 데이터 나누기
    both_1 = full_df[(full_df['rater1'] == 1) & (full_df['rater2'] == 1)].copy()
    both_0 = full_df[(full_df['rater1'] == 0) & (full_df['rater2'] == 0)].copy()
    disagreement = full_df[((full_df['rater1'] == 1) & (full_df['rater2'] == 0)) |
                         ((full_df['rater1'] == 0) & (full_df['rater2'] == 1))].copy()

    # CSV 파일로 저장
    both_1_path = f'{output_prefix}both_1.csv'
    both_0_path = f'{output_prefix}both_0.csv'
    disagreement_path = f'{output_prefix}disagreement.csv'

    both_1.to_csv(both_1_path, index=False)
    both_0.to_csv(both_0_path, index=False)
    disagreement.to_csv(disagreement_path, index=False)

    # 생성된 CSV 파일 압축
    if create_zip:
        zip_filename = f'{output_prefix}agreement_results.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in [both_1_path, both_0_path, disagreement_path]:
                zipf.write(file, os.path.basename(file))
        print(f"CSV 파일들이 {zip_filename}으로 압축되었습니다.\n")

    # 통계 결과 반환
    results = {
        "총_데이터_개수": len(full_df),
        "둘_다_1인_행": len(both_1),
        "둘_다_0인_행": len(both_0),
        "불일치_행": len(disagreement),
        "코헨_카파": stats['kappa'],
        "관찰된_일치도": stats['observed_agreement'],
        "우연에_의한_일치도": stats['chance_agreement']
    }

    return results, stats


path1 = "/content/drive/MyDrive/fin/ABSA/사회_단어_jace.csv"
path2 = "/content/drive/MyDrive/fin/ABSA/사회_단어_rodri.csv"

results, stats = analyze_agreement(path1, path2, output_prefix="사회_", create_zip=True) # 경제_ 환경_ 행정_

# 기본 결과 출력
print(f"총 데이터 개수: {results['총_데이터_개수']}")
print(f"둘 다 1인 행: {results['둘_다_1인_행']}개")
print(f"둘 다 0인 행: {results['둘_다_0인_행']}개")
print(f"불일치 행: {results['불일치_행']}개")

# 카파 통계 결과 출력
print(f"\nCohen's Kappa: {stats['kappa']:.4f}")
print(f"관찰된 일치도: {stats['observed_agreement']:.4f}")
print(f"우연에 의한 일치도: {stats['chance_agreement']:.4f}")
print(f"직접 계산한 Kappa: {stats['manual_kappa']:.4f}")

# 고유 값 출력
print("\n고유 값:", stats['unique_values'])

# 각 값의 비율 출력
for val, ratios in stats['value_ratios'].items():
    print(f"값 {val}: 평가자1 비율 {ratios['rater1']:.4f}, 평가자2 비율 {ratios['rater2']:.4f}")


# 압축 파일 다운로드
from google.colab import files
files.download(f'사회_agreement_results.zip') # 입력

CSV 파일들이 사회_agreement_results.zip으로 압축되었습니다.

총 데이터 개수: 2263
둘 다 1인 행: 1898개
둘 다 0인 행: 287개
불일치 행: 78개

Cohen's Kappa: 0.8603
관찰된 일치도: 0.9655
우연에 의한 일치도: 0.7533
직접 계산한 Kappa: 0.8603

고유 값: [0. 1.]
값 0.0: 평가자1 비율 0.1379, 평가자2 비율 0.1502
값 1.0: 평가자1 비율 0.8621, 평가자2 비율 0.8498


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>